# 累计/聚合与分组(Aggregation-and-Grouping)
在对较大的数据进行分析时，一项基本的工作就是有效的数据累计（summarization）：计 算累计（aggregation）指标，如sum()、mean()、median()、min() 和max()，其中每一个指 标都呈现了大数据集的特征。在这一节中，我们将探索Pandas 的累计功能，从类似前面 NumPy 数组中的简单操作，到基于groupby 实现的复杂操作。

For convenience, we'll use the same display magic function that we've seen in previous sections:

In [1]:
import numpy as np
import pandas as pd

class display(object):
    """Display HTML representation of multiple objects"""
    template = """<div style="float: left; padding: 10px;">
    <p style='font-family:"Courier New", Courier, monospace'>{0}</p>{1}
    </div>"""
    def __init__(self, *args):
        self.args = args
        
    def _repr_html_(self):
        return '\n'.join(self.template.format(a, eval(a)._repr_html_())
                         for a in self.args)
    
    def __repr__(self):
        return '\n\n'.join(a + '\n' + repr(eval(a))
                           for a in self.args)

## 行星数据(Planets Data)
我们将通过Seaborn 程序库（http://seaborn.pydata.org）用一份行星数据来进行演示，其中包含天文学家观测到的围绕恒星运转的行星数据（通常简称为太阳系外行星或外行星）。行星数据可以直接通过Seaborn 下载：

In [2]:
import seaborn as sns
planets = sns.load_dataset('planets')
planets.shape

(1035, 6)

In [3]:
type(planets)

pandas.core.frame.DataFrame

In [4]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


数据中包含了截至2014 年已被发现的一千多颗外行星的资料。

## Pandas的简单累计功能(Simple Aggregation in Pandas)
之前我们介绍过NumPy 数组的一些数据累计指标。与一维NumPy 数组相同，Pandas 的Series 的累计函数也会返回一个统计值：

In [5]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.rand(5))
ser

0    0.374540
1    0.950714
2    0.731994
3    0.598658
4    0.156019
dtype: float64

In [6]:
ser.sum()

2.811925491708157

In [7]:
ser.mean()

0.5623850983416314

**DataFrame 的累计函数默认对每列进行统计**：

In [8]:
df = pd.DataFrame({'A': rng.rand(5),
                   'B': rng.rand(5)})
df

,A,B
0,0.155995,0.020584
1,0.058084,0.969910
2,0.866176,0.832443
3,0.601115,0.212339
4,0.708073,0.181825


In [9]:
df.mean()

A    0.477888
B    0.443420
dtype: float64

**设置axis 参数，你就可以对每一行进行统计了**：

In [10]:
df.mean(axis='columns')

0    0.088290
1    0.513997
2    0.849309
3    0.406727
4    0.444949
dtype: float64

Pandas 的Series 和DataFrame 支持所有numpy中的常用累计函数。另外，还有一个非常方便的describe() 方法可以计算每一列的若干常用统计值。让我们在行星数据上试验一下，首先丢弃有缺失值的行：

In [11]:
planets.dropna().describe()

,number,orbital_period,mass,distance,year
count,498.00000,498.000000,498.000000,498.000000,498.000000
mean,1.73494,835.778671,2.509320,52.068213,2007.377510
std,1.17572,1469.128259,3.636274,46.596041,4.167284
min,1.00000,1.328300,0.003600,1.350000,1989.000000
25%,1.00000,38.272250,0.212500,24.497500,2005.000000
50%,1.00000,357.000000,1.245000,39.940000,2009.000000
75%,2.00000,999.600000,2.867500,59.332500,2011.000000
max,6.00000,17337.500000,25.000000,354.000000,2014.000000


这是一种理解数据集所有统计属性的有效方法。例如，从年份year 列中可以看出，1989年首次发现外行星，而且一半的已知外行星都是在2010 年及以后的年份被发现的。这主要得益于开普勒计划——一个通过激光望远镜发现恒星周围椭圆轨道行星的太空计划。

In [12]:
planets.describe()

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


Pandas 内置的一些累计方法如表所示。

| 指标             | 描述                                    |
| ---------------- | --------------------------------------- |
| count()          | 计数项                                  |
| first()、last()  | 第一项与最后一项                        |
| mean()、median() | 均值与中位数                            |
| min()、max()     | 最小值与最大值                          |
| std()、var()     | 标准差与方差                            |
| mad()            | 均值绝对偏差（mean absolute deviation） |
| prod()           | 所有项乘积                              |
| sum()            | 所有项求和                              |

DataFrame 和Series 对象支持以上所有方法。

但若想深入理解数据，仅仅依靠累计函数是远远不够的。数据累计的下一级别是groupby操作，它可以让你快速、有效地计算数据各子集的累计值。

## GroupBy：分割、应用和组合(GroupBy: Split, Apply, Combine)
简单的累计方法可以让我们对数据集有一个笼统的认识，但是我们经常还需要对某些标签或索引的局部进行累计分析，这时就需要用到groupby 了。虽然“分组”（group by）这个名字是借用SQL 数据库语言的命令，但其理念引用发明R 语言frame 的Hadley Wickham的观点可能更合适：分割（split）、应用（apply）和组合（combine）。

#### 1. 分割、应用和组合
一个经典分割- 应用- 组合操作示例如下图所示，其中“apply”的是一个求和函数。

下图清晰地描述了GroupBy 的过程：  
* 分割步骤将DataFrame 按照指定的键分割成若干组。  
* 应用步骤对每个组应用函数，通常是累计、转换或过滤函数。  
* 组合步骤将每一组的结果合并成一个输出数组。  

![2019-04-14_224704.png](imgs/2019-04-14_224704.png)

虽然我们也可以通过前面介绍的一系列的掩码、累计与合并操作来实现，但是意识到中间分割过程不需要显式地暴露出来这一点十分重要。而且GroupBy（经常）只需要一行代码，就可以计算每组的和、均值、计数、最小值以及其他累计值。GroupBy 的用处就是将这些步骤进行抽象：用户不需要知道在底层如何计算，只要把操作看成一个整体就够了。

用Pandas 进行上图所示的计算作为具体的示例。从创建输入DataFrame 开始：

In [13]:
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data': range(6)}, columns=['key', 'data'])
df

,key,data
0,A,0
1,B,1
2,C,2
3,A,3
4,B,4
5,C,5


我们可以用DataFrame 的groupby() 方法进行绝大多数常见的分割- 应用- 组合操作，将需要分组的列名传进去即可：

In [14]:
df.groupby('key')

需要注意的是，这里的返回值不是一个DataFrame 对象，而是一个DataFrameGroupBy 对象。这个对象的魔力在于，你可以将它看成是一种特殊形式的DataFrame，里面隐藏着若干组数据，但是在没有应用累计函数之前不会计算。这种“延迟计算”（lazy evaluation）的方法使得大多数常见的累计操作可以通过一种对用户而言几乎是透明的（感觉操作仿佛不存在）方式非常高效地实现。

为了得到这个结果，可以对DataFrameGroupBy 对象应用累计函数，它会完成相应的应用/组合步骤并生成结果：

In [15]:
df.groupby('key').sum()

,data
key,
A,3
B,5
C,7


sum() 只是众多可用方法中的一个。你可以用Pandas 或NumPy 的任意一种累计函数，也可以用任意有效的DataFrame 对象。下面就会介绍。

#### 2. GroupBy对象
GroupBy 对象是一种非常灵活的抽象类型。在大多数场景中，你可以将它看成是DataFrame的集合，在底层解决所有难题。让我们用行星数据来做一些演示。

GroupBy 中最重要的操作可能就是aggregate、filter、transform 和apply（累计、过滤、转换、应用）了，后文将详细介绍这些内容，现在先来介绍一些GroupBy 的基本操作方法。

###### (1) 按列取值。
GroupBy 对象与DataFrame 一样，也支持按列取值，并返回一个修改过的GroupBy 对象，例如：

In [16]:
planets.groupby('method')

In [17]:
planets.groupby('method')['orbital_period']

这里从原来的DataFrame 中取某个列名作为一个Series 组。与GroupBy 对象一样，直到我们运行累计函数，才会开始计算：

In [18]:
planets.groupby('method')['orbital_period'].median()

method
Astrometry                         631.180000
Eclipse Timing Variations         4343.500000
Imaging                          27500.000000
Microlensing                      3300.000000
Orbital Brightness Modulation        0.342887
Pulsar Timing                       66.541900
Pulsation Timing Variations       1170.000000
Radial Velocity                    360.200000
Transit                              5.714932
Transit Timing Variations           57.011000
Name: orbital_period, dtype: float64

这样就可以获得不同方法下所有行星公转周期（按天计算）的中位数。

###### (2) 按组迭代。
GroupBy 对象支持直接按组进行迭代，返回的每一组都是Series 或DataFrame：

In [19]:
for (method, group) in planets.groupby('method'):
    print("{0:30s} shape={1}".format(method, group.shape))

Astrometry                     shape=(2, 6)
Eclipse Timing Variations      shape=(9, 6)
Imaging                        shape=(38, 6)
Microlensing                   shape=(23, 6)
Orbital Brightness Modulation  shape=(3, 6)
Pulsar Timing                  shape=(5, 6)
Pulsation Timing Variations    shape=(1, 6)
Radial Velocity                shape=(553, 6)
Transit                        shape=(397, 6)
Transit Timing Variations      shape=(4, 6)


尽管通常还是使用内置的apply 功能速度更快，但这种方式在手动处理某些问题时非常有用，后面会详细介绍。

###### (3) 调用方法。
借助Python 类的魔力（@classmethod），可以让任何不由GroupBy 对象直接实现的方法直接应用到每一组，无论是DataFrame 还是Series 对象都同样适用。

例如，你可以用DataFrame 的describe() 方法进行累计，对每一组数据进行描述性统计：

In [20]:
planets.groupby('method')['year'].describe()

,count,mean,std,min,25%,50%,75%,max
method,,,,,,,,
Astrometry,2.0,2011.500000,2.121320,2010.0,2010.75,2011.5,2012.25,2013.0
Eclipse Timing Variations,9.0,2010.000000,1.414214,2008.0,2009.00,2010.0,2011.00,2012.0
Imaging,38.0,2009.131579,2.781901,2004.0,2008.00,2009.0,2011.00,2013.0
Microlensing,23.0,2009.782609,2.859697,2004.0,2008.00,2010.0,2012.00,2013.0
Orbital Brightness Modulation,3.0,2011.666667,1.154701,2011.0,2011.00,2011.0,2012.00,2013.0
Pulsar Timing,5.0,1998.400000,8.384510,1992.0,1992.00,1994.0,2003.00,2011.0
Pulsation Timing Variations,1.0,2007.000000,NaN,2007.0,2007.00,2007.0,2007.00,2007.0
Radial Velocity,553.0,2007.518987,4.249052,1989.0,2005.00,2009.0,2011.00,2014.0
Transit,397.0,2011.236776,2.077867,2002.0,2010.00,2012.0,2013.00,2014.0


这张表可以帮助我们对数据有更深刻的认识，例如大多数行星都是通过Radial Velocity和Transit 方法发现的，而且后者在近十年变得越来越普遍（得益于更新、更精确的望远镜）。最新的Transit Timing Variation 和Orbital Brightness Modulation 方法在2011 年之后才有新的发现。

这只是演示Pandas 调用方法的示例之一。方法首先会应用到每组数据上，然后结果由GroupBy 组合后返回。另外，任意DataFrame / Series 的方法都可以由GroupBy 方法调用，从而实现非常灵活强大的操作。

#### 3. 累计、过滤、转换和应用(Aggregate, filter, transform, apply)
虽然前面的章节只重点介绍了组合操作，但是还有许多操作没有介绍，尤其是GroupBy 对象的aggregate()、filter()、transform() 和apply() 方法，在数据组合之前实现了大量高效的操作。

为了方便后面内容的演示，使用下面这个DataFrame：

In [21]:
rng = np.random.RandomState(0)
df = pd.DataFrame({'key': ['A', 'B', 'C', 'A', 'B', 'C'],
                   'data1': range(6),
                   'data2': rng.randint(0, 10, 6)},
                   columns = ['key', 'data1', 'data2'])
df

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9


###### (1) 累计。
我们目前比较熟悉的GroupBy 累计方法只有sum() 和median() 之类的简单函数，但是aggregate() 其实可以支持更复杂的操作，比如字符串、函数或者函数列表，并且能一次性计算所有累计值。

下面来快速演示一个例子：

In [22]:
df.groupby('key').aggregate(['min', np.median, max])

data1            data2           
      min median max   min median max
key                                  
A       0    1.5   3     3    4.0   5
B       1    2.5   4     0    3.5   7
C       2    3.5   5     3    6.0   9

另一种用法就是通过Python 字典指定不同列需要累计的函数：

In [23]:
df.groupby('key').aggregate({'data1': 'min',
                             'data2': 'max'})

,data1,data2
key,,
A,0,5
B,1,7
C,2,9


###### (2) 过滤。
过滤操作可以让你按照分组的属性丢弃若干数据。

例如，我们可能只需要保留标准差超过某个阈值的组：

In [24]:
def filter_func(x):
    return x['data2'].std() > 4

display('df', "df.groupby('key').std()", "df.groupby('key').filter(filter_func)")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby('key').std()
       data1     data2
key                   
A    2.12132  1.414214
B    2.12132  4.949747
C    2.12132  4.242641

df.groupby('key').filter(filter_func)
  key  data1  data2
1   B      1      0
2   C      2      3
4   B      4      7
5   C      5      9

filter() 函数会返回一个布尔值，表示每个组是否通过过滤。由于A 组'data2' 列的标准差不大于4，所以被丢弃了。

###### (3) 转换。
累计操作返回的是对组内全量数据缩减过的结果，而转换操作会返回一个新的全量数据。数据经过转换之后，其形状与原来的输入数据是一样的。常见的例子就是将每一组的样本数据减去各组的均值，实现数据标准化：

In [25]:
df.groupby('key').transform(lambda x: x - x.mean())

,data1,data2
0,-1.5,1.0
1,-1.5,-3.5
2,-1.5,-3.0
3,1.5,-1.0
4,1.5,3.5
5,1.5,3.0


###### (4) apply() 方法。
apply() 方法让你可以在每个组上应用任意方法。这个函数输入一个DataFrame，返回一个Pandas 对象（DataFrame 或Series）或一个标量（scalar，单个数值）。组合操作会适应返回结果类型。

下面的例子就是用apply() 方法将第一列数据以第二列的和为基数进行标准化：

In [26]:
def norm_by_data2(x):
    # x是一个分组数据的DataFrame
    # x is a DataFrame of group values
    x['data1'] /= x['data2'].sum()
    return x

display('df', "df.groupby('key').apply(norm_by_data2)")

,key,data1,data2
0,A,0,5
1,B,1,0
2,C,2,3
3,A,3,3
4,B,4,7
5,C,5,9
,key,data1,data2
0,A,0.000000,5
1,B,0.142857,0
2,C,0.166667,3


GroupBy 里的apply() 方法非常灵活，唯一需要注意的地方是它总是输入分组数据的DataFrame，返回Pandas 对象或标量。具体如何选择需要视情况而定。

#### 4. 设置分割的键
前面的简单例子一直在用列名分割DataFrame。这只是众多分组操作中的一种，下面将继续介绍更多的分组方法。

###### (1) 将列表、数组、Series 或索引作为分组键。
分组键可以是长度与DataFrame 匹配的任意Series 或列表，例如：

In [27]:
L = [0, 1, 0, 1, 2, 0]
display('df', 'df.groupby(L).sum()')

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(L).sum()
   data1  data2
0      7     17
1      4      3
2      4      7

因此，还有一种比前面直接用列名更啰嗦的表示方法df.groupby('key')：

In [28]:
display('df', "df.groupby(df['key']).sum()")

df
  key  data1  data2
0   A      0      5
1   B      1      0
2   C      2      3
3   A      3      3
4   B      4      7
5   C      5      9

df.groupby(df['key']).sum()
     data1  data2
key              
A        3      8
B        5      7
C        7     12

###### (2) 用字典或Series 将索引映射到分组名称。

另一种方法是提供一个字典，将索引映射到分组键：

In [29]:
df2 = df.set_index('key')
mapping = {'A': 'vowel', 'B': 'consonant', 'C': 'consonant'}
display('df2', 'df2.groupby(mapping).sum()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
consonant,12,19
vowel,3,8


In [30]:
df2

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9


###### (3) 任意Python 函数。
与前面的字典映射类似，你可以将任意Python 函数传入groupby，函数映射到索引，然后新的分组输出：

In [31]:
display('df2', 'df2.groupby(str.lower).mean()')

,data1,data2
key,,
A,0,5
B,1,0
C,2,3
A,3,3
B,4,7
C,5,9
,data1,data2
a,1.5,4.0
b,2.5,3.5


###### (4) 多个有效键构成的列表。
此外，任意之前有效的键都可以组合起来进行分组，从而返回一个多级索引的分组结果：

In [32]:
df2.groupby([str.lower, mapping]).mean()

,,data1,data2
a,vowel,1.5,4.0
b,consonant,2.5,3.5
c,consonant,3.5,6.0


#### 5. 分组案例
通过下例中的几行Python 代码，我们就可以运用上述知识，获取不同方法和不同年份发现的行星数量：

In [33]:
decade = 10 * (planets['year'] // 10)
decade = decade.astype(str) + 's'
decade.name = 'decade'
planets.groupby(['method', decade])['number'].sum().unstack().fillna(0)

decade,1980s,1990s,2000s,2010s
method,,,,
Astrometry,0.0,0.0,0.0,2.0
Eclipse Timing Variations,0.0,0.0,5.0,10.0
Imaging,0.0,0.0,29.0,21.0
Microlensing,0.0,0.0,12.0,15.0
Orbital Brightness Modulation,0.0,0.0,0.0,5.0
Pulsar Timing,0.0,9.0,1.0,1.0
Pulsation Timing Variations,0.0,0.0,1.0,0.0
Radial Velocity,1.0,52.0,475.0,424.0
Transit,0.0,0.0,64.0,712.0


In [34]:
planets.groupby(['method', decade])['number'].sum()

method                         decade
Astrometry                     2010s       2
Eclipse Timing Variations      2000s       5
                               2010s      10
Imaging                        2000s      29
                               2010s      21
Microlensing                   2000s      12
                               2010s      15
Orbital Brightness Modulation  2010s       5
Pulsar Timing                  1990s       9
                               2000s       1
                               2010s       1
Pulsation Timing Variations    2000s       1
Radial Velocity                1980s       1
                               1990s      52
                               2000s     475
                               2010s     424
Transit                        2000s      64
                               2010s     712
Transit Timing Variations      2010s       9
Name: number, dtype: int64

此例足以展现GroupBy 在探索真实数据集时快速组合多种操作的能力——只用寥寥几行代码，就可以让我们立即对过去几十年里不同年代的行星发现方法有一个大概的了解。

我建议你花点时间分析这几行代码，确保自己真正理解了每一行代码对结果产生了怎样的影响。虽然这个例子的确有点儿复杂，但是理解这几行代码的含义可以帮你掌握分析类似数据的方法。